In [2]:
from PIL import Image
import numpy as np
import pandas as pd

import torch

from script.tool import ROOT, ROOT_NFS_TEST, ROOT_NFS_DATA, standardize_feature, convert_feature
from tqdm import tqdm
from pathlib import Path
import time, os

In [3]:
path_dataset = ROOT_NFS_DATA / 'Cosmenet_product_20231018'
device = torch.device("cuda:0")
df_pd = pd.read_csv(path_dataset / 'datas_20231018.csv')
df_pd.head(1)

,file_names,labels,images_path
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...


In [4]:
group_df = df_pd.groupby(['labels'])['labels'].count().reset_index(name='count').sort_values(['count'], ascending=False)
group_df.head(1)

,labels,count
4172,50348,100


In [5]:
two_image = group_df[group_df['count'] <= 2]['labels'].values
df_not_two = df_pd[~df_pd['labels'].isin(two_image)]

In [6]:
print(f"amount of all data : {df_pd.__len__()}")
print(f"amount of data without two image : {df_not_two.__len__()}")
print(f"amount of data two image : {df_pd[df_pd['labels'].isin(two_image)].__len__()}")

amount of all data : 60196
amount of data without two image : 60131
amount of data two image : 65


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

In [8]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

In [9]:
train_idx, test_idx = sss.split(df_not_two, df_not_two['labels']).__next__()
df_train = df_not_two.iloc[train_idx]
df_test = df_not_two.iloc[test_idx]

In [10]:
print(f"amount of all class : {df_pd['labels'].nunique()}")
print(f"amount of class two img : {two_image.__len__()}")
print(f"amount of train class : {df_not_two['labels'].iloc[train_idx].nunique()}")
print(f"amount of test class : {df_not_two['labels'].iloc[test_idx].nunique()}")

amount of all class : 4178
amount of class two img : 41
amount of train class : 4137
amount of test class : 4137


In [11]:
def select_transformers_model(model, processor, pretrain="google/vit-base-patch16-224-in21k"):
    model = model.from_pretrained(pretrain)
    processor = processor.from_pretrained(pretrain)
    return model, processor

In [12]:
# pipeline for transformer library
class pipeline_transformer:
    def __init__(self, layer, row=False, device='cuda:0'):
        self.device = device
        self.layer = layer
        self.row = row
    
    def selct_model(self, model, processor):
        self.model = model
        self.processor = processor
        self.model.eval().to(self.device)
    
    def process_model(self, img):
        inputs = self.processor(images=img, return_tensors="pt").to(self.device)
        outputs = self.model(**inputs)
        return outputs
        
    def extract(self, img):
        ### return specific layer
        outputs = self.process_model(img)
        if type(self.row) == bool and not self.row:
            outputs = outputs[self.layer]
        else:
            outputs = outputs[self.layer][:, self.row]
        outputs = outputs.flatten().unsqueeze(0)
        outputs = standardize_feature(outputs).to('cpu').detach().numpy()
        return outputs
    
    def report_test(self):
        img = Image.new('RGB', (224, 224))
        start_time_torch = time.time()
        outputs = self.process_model(img)
        delta_time_torch = time.time() - start_time_torch
        print("runtime :", delta_time_torch*1000, "ms")
        print(f"outputs layers : {outputs.keys()}")
        print(f"shape last_hidden_state : {outputs.last_hidden_state.shape}")
        print(f"shape pooler_output : {outputs.pooler_output.shape}")

In [13]:
from transformers import ViTImageProcessor, ViTModel

In [15]:
model, preprocess = select_transformers_model(ViTModel, ViTImageProcessor, 
                                              pretrain=ROOT_NFS_TEST / 'weights/vit_gg_lr2e-05_eu_9ep_0_95099acc')
vit_gg_trained_lr2e_05_pipe = pipeline_transformer(layer="last_hidden_state", row=0, device=device)
vit_gg_trained_lr2e_05_pipe.selct_model(model, preprocess)
vit_gg_trained_lr2e_05_pipe.report_test()

runtime : 11.174440383911133 ms
outputs layers : odict_keys(['last_hidden_state', 'pooler_output'])
shape last_hidden_state : torch.Size([1, 197, 768])
shape pooler_output : torch.Size([1, 768])


In [16]:
print(df_train.head(1))
print(df_test.head(1))

        file_names  labels                                        images_path
34244  40959_8.png   40959  /app/nfs_clientshare/Datasets/Cosmenet_product...
        file_names  labels                                        images_path
41380  46687_6.png   46687  /app/nfs_clientshare/Datasets/Cosmenet_product...


In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es = Elasticsearch(HOST="http://localhost", PORT=9200)

In [ ]:
if es.indices.exists(index="test_product"):
    print("index already exists")
else:
    body_product = {
        "mappings":{  
            "product":{  
                "_timestamp":{  
                    "enabled":"true"
                },
                "properties":{
                    "labels":{
                        "type":"text"
                    },
                    "file_names":{
                        "type":"text"
                    },
                    "images_path":{
                        "type":"text"
                    },
                    "features":{  
                        "type":"dense_vector",
                        "dims":768
                    },
                    
                }
            }
        }
    }
    es.indices.create(index="test_product", body=body_product, ignore=400)

In [17]:
for n, img_path in enumerate(tqdm(df_train['images_path'])):
    img = Image.open(img_path).convert('RGB')
    output = vit_gg_trained_lr2e_05_pipe.extract(img).flatten()
    data = {
        "labels": df_train['labels'].iloc[n],
        "file_names": df_train['file_names'].iloc[n],
        "images_path": img_path,
        "features": output,
    }
    # es.index(index="test_product", doc_type="product", id=n, body=data)
    if n == 10:
        break

  0%|          | 10/48104 [00:00<51:14, 15.65it/s]
